# Sesión 7.4 Cargar y trabajar con modelos con fine tuning de Huggingface

Existen cientos de modelos ya con fine tuning ya realizado para distintas tareas en Huggingface (https://huggingface.co/) tanto de modelos BERT/RoBERTa como otros modelos de transformers como GPT.

Estos modelos no son únicamente para tareas de PLN sino que hay otros modelos de procesamiento de imágenes, etc.

En esta sesión vamos a cargar y probar 3 modelos para distintas tareas de PLN, en concreto: análisis de sentimientos, detección de entidades (NER) y resumen.

1.  Instalamos las librerías de transformers y datasets.
2.  Descargamos un modelo de análisis de sentimientos en español.
3.  Definimos un conjunto de textos, los tokenizamos, y se los introducimos al modelo para que los clasifique.
4.  Procedemos de manera semejante con otro modelo, en este caso para realizar reconocimiento de entidades nombradas (NER).
5.  Utilizamos un modelo para resumen de texto.

In [1]:
# Install libraries
!pip3 install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

## Apartado 1.1 Descargamos y probamos un modelo de análisis de sentimientos en español

El modelo en cuestión es el que se encuentra en la página https://huggingface.co/finiteautomata/beto-sentiment-analysis y es un finetuning de un modelo BETO usando el dataset del TASS 2020.

In [2]:
import transformers
from transformers import AutoModelForSequenceClassification

path_sentiment_model = 'finiteautomata/beto-sentiment-analysis'

# Obtenemos el tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained (path_sentiment_model)
# Obtenemos el modelo para análisis de sentimientos que tiene 3 etiquetas: positivo, negativo y neutro
sentiment_model = AutoModelForSequenceClassification.from_pretrained(path_sentiment_model)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/481k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

# Apartado 1.2 Probamos el modelo de análisis de sentimientos con un conjunto de textos.

Obtenemos la clasificación del sentimiento a partir de un conjunto de textos.

Estos modelos a los que ya se les ha hecho un finetuning se pueden volver a entrenar con otros finetuning.

In [3]:
import torch

# Probamos a clasificar estas frases
textos = ['la película es muy mala y los actores no son los adecuados',
          'me encanta este actor',
          'la pelicula dura 2 horas',
          'estoy encantado con el regalo que me han hecho',
          'hay muchos más muertos por covid',
          'el número de afectados por covid aumenta',
          'vamos a salir de la pandemia',
          'ánimo a todos'
]

# Se obtienen los sentimientos de los textos
for text in textos:
  inputs = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    logits = sentiment_model(**inputs).logits
  predicted_class_id = logits.argmax().item()
  prediction= sentiment_model.config.id2label[predicted_class_id]

  print(text,'=>', prediction)


la película es muy mala y los actores no son los adecuados => NEG


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

me encanta este actor => POS
la pelicula dura 2 horas => NEU
estoy encantado con el regalo que me han hecho => POS
hay muchos más muertos por covid => NEG
el número de afectados por covid aumenta => NEG
vamos a salir de la pandemia => NEU
ánimo a todos => POS


## Apartado 1.3 Probamos ahora un modelo de NER en español

A continuación probamos el modelo de reconocimiento de entidades en español NER (https://huggingface.co/PlanTL-GOB-ES/roberta-base-bne-capitel-ner-plus) entrenado con el dataset de CAPITEL

In [4]:
spanish_ner_model = 'PlanTL-GOB-ES/roberta-base-bne-capitel-ner-plus'

from transformers import AutoModelForTokenClassification


# Obtenemos el tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained (spanish_ner_model)
# Obtenemos el modelo para la extracción de entidades
ner_model = AutoModelForTokenClassification.from_pretrained(spanish_ner_model)


tokenizer_config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/496M [00:00<?, ?B/s]

# Apartado 1.4 Probamos como extraer entidades de distintos textos

Probamos cómo puede extraer las entidades de distintos textos según los tokens obtenidos por el tokenizer de BERT.

In [5]:
from transformers import pipeline

nlp = pipeline("ner", model=ner_model, tokenizer=tokenizer)
examples = ["Soy Juan Pastor profesor de la asignatura PLNE del Grado en Ciencias en Ingeniería de Datos en la UPCT.",
            "Carlos Alcaraz es un tenista de Murcia que está destacando en las competiciones ATP",
            "Mercadona es una cadena de supermercados que está en toda España"
]

ner_results = nlp(examples)
for ner_text, i in zip(ner_results, range(len(ner_results ))):
  print(examples[i])
  for ner in ner_text:
    print(ner)

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

Device set to use cpu


Soy Juan Pastor profesor de la asignatura PLNE del Grado en Ciencias en Ingeniería de Datos en la UPCT.
{'entity': 'B-PER', 'score': np.float32(0.9997454), 'index': 2, 'word': 'ĠJuan', 'start': 4, 'end': 8}
{'entity': 'E-PER', 'score': np.float32(0.9997662), 'index': 3, 'word': 'ĠPastor', 'start': 9, 'end': 15}
{'entity': 'S-OTH', 'score': np.float32(0.998823), 'index': 8, 'word': 'ĠPL', 'start': 42, 'end': 44}
{'entity': 'S-OTH', 'score': np.float32(0.92223674), 'index': 9, 'word': 'NE', 'start': 44, 'end': 46}
{'entity': 'S-OTH', 'score': np.float32(0.5829536), 'index': 11, 'word': 'ĠGrado', 'start': 51, 'end': 56}
{'entity': 'I-OTH', 'score': np.float32(0.6618023), 'index': 13, 'word': 'ĠCiencias', 'start': 60, 'end': 68}
{'entity': 'I-OTH', 'score': np.float32(0.8538995), 'index': 15, 'word': 'ĠIngenierÃŃa', 'start': 72, 'end': 82}
{'entity': 'I-OTH', 'score': np.float32(0.9937605), 'index': 16, 'word': 'Ġde', 'start': 83, 'end': 85}
{'entity': 'E-OTH', 'score': np.float32(0.990310

## Apartado 1.5 Modelos para resumen de texto
Probamos un ejemplo de resumidor de texto basado en modelos RoBERTa
https://huggingface.co/Narrativa/bsc_roberta2roberta_shared-spanish-finetuned-mlsum-summarization

In [6]:
import torch
from transformers import RobertaTokenizerFast, EncoderDecoderModel
device = 'cuda' if torch.cuda.is_available() else 'cpu'
ckpt = 'Narrativa/bsc_roberta2roberta_shared-spanish-finetuned-mlsum-summarization'
tokenizer = RobertaTokenizerFast.from_pretrained(ckpt)
model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

def generate_summary(text):

   inputs = tokenizer([text], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
   input_ids = inputs.input_ids.to(device)
   attention_mask = inputs.attention_mask.to(device)
   output = model.generate(input_ids, attention_mask=attention_mask)
   return tokenizer.decode(output[0], skip_special_tokens=True)

text = """BONO TURÍSTICO DE LA REGIÓN DE MURCIA: ASÍ PUEDES CONSEGUIR EL 50%
DE DESCUENTO

La Consejería de Turismo, Cultura, Juventud y Deportes aprobó
recientemente los nuevos bonos turísticos con descuentos en
alojamientos de la Región de Murcia. La iniciativa, que se activó el
pasado 12 de noviembre y permanecerá vigente hasta el 31 de mayo de
2024, ofrecerá ayudas directas para cubrir el coste de los servicios
subvencionables del viaje.

Esta tercera edición busca, como las anteriores de 2021 y 2022,
promover la llegada de turistas a la Comunidad durante todo el año.
En esta ocasión, es la primera vez que han optado por iniciarla en la
temporada de otoño-invierno. Con este objetivo, se ha puesto en
circulación un total de 2.500 bonos, que se otorgarán por orden de
formalización de reservas.

Este bono ofrecerá a sus beneficiarios un descuento del 50%, con un
máximo de 250 euros (IVA incluido). Para solicitarlo se deben
contratar unos servicios mínimos, que corresponden a una estancia de,
al menos, dos noches consecutivas en el mismo alojamiento adherido al
programa.

QUIÉN PUEDE SOLICITARLO

Estas ayudas van dirigidas a cualquier persona mayor de edad con
residencia legal y continuada en España. También está contemplado
para ciudadanos de la Región de Murcia. Cada viajero podrá solicitar
más de un bono dentro de cada uno de los periodos de vigencia y
deberá haber pasado un total de diez días naturales desde el momento
en el que realizó la última solicitud. Los viajes no podrán tener
carácter profesional.

Estos periodos de vigencia se dividen en dos. El primero, que parte
desde el día posterior a la publicación de la convocatoria en el
BORM hasta el 22 de diciembre de 2023. Por su parte, el segundo está
activo desde el día siguiente a la publicación en el BORM y hasta el
31 de mayo de 2024. Eso sí, la subvención debe canjearse en el mismo
periodo en el que se adquiere.

CÓMO SE SOLICITA

En la página web www.turismoregiondemurcia.es/es/bono_turistico se
encuentra un apartado llamado 'adquiere tu bono'. Al acceder, se
desplegará un formulario de inscripción, en el que se deben rellenar
todos los datos personales de la persona beneficiaria y completar una
encuesta sobre la intención del viaje. Asimismo, en este documento se
deberá aceptar el contenido de la declaración responsable, que se
podrá descargar y rellenar en un enlace que se aporta dentro de este
mismo formulario.

Una vez que se hayan completado todos los campos, se pulsará sobre
'registrarme'. Acto seguido, recibirá un correo electrónico con un
enlace de confirmación. Este paso es fundamental, ya que de no
confirmar ese registro, no se podrá acceder a las ventajas de la
ayuda.

Al validar la inscripción, recibirá otro correo con el código
promocional o el número de la lista de espera (si se da la
circunstancia de que los bonos se han agotado). Cabe recordar que si
no se formaliza una reserva en un plazo de diez días naturales desde
su adquisición, este caducará y deberá volver a solicitarse.
Además, desde la fecha en la que el alojamiento confirme la reserva
hasta el primer día en el que se disfrute el servicio, deberá
existir un mínimo de siete días naturales y un máximo de 20.

QUÉ SERVICIOS SON SUBVENCIONABLES

Las estancias que forman parte del programa se encuentran en la
tipología de establecimientos hoteleros, apartamentos turísticos,
campings, casas rurales, hospederías rurales y albergues. En la web
oficial de turismo de Murcia se encuentran todos los adscritos a esta
iniciativa.

Los servicios subvencionados que están contemplados en esta ayuda son
los siguientes:

- Manutención, como máximo en régimen de desayuno y con exclusión
de bebidas alcohólicas.

· Tratamientos de salud y bienestar del alojamiento.

· Alquiler de instalaciones y/o equipamiento deportivo del
establecimiento.

· Actividades de animación o deportivas prestadas por el personal
del establecimiento.

- Aparcamiento."""

generate_summary(text)

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.62k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/615M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.roberta.modeling_roberta.RobertaModel'> is overwritten by shared encoder config: RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.50.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50262
}

Config of the decoder: <class 'transformers.models.roberta.modeling_roberta.RobertaForCausalLM'> is overwritten by shared decoder config: RobertaConfig {
  "add_cross_attention": tr

'La Consejería de Cultura, Cultura y Juventud de la Comunidad de Murcia ofrece 2.500 bonos turísticos con descuentos del 50%'

## Apartado 1.6 Modelos para otras tareas

Podéis probar otros modelos para otras tareas distintas consultando cómo se implementaría la llamada a los mismos https://huggingface.co/transformers/usage.html

In [8]:
# Voy a probar con dos modelos diferentes

# 1) Traducción de texto
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Cargamos el modelo y el tokenizer
model_name = 'Helsinki-NLP/opus-mt-en-es' # Modelo para traducir de inglés a español
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Texto de entrada
text = 'Hugging Face is creating amazing tools for NLP.'

# Tokenización
inputs = tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True)

# Traducción
outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Texto traducido: {translated_text}")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Texto traducido: Hugging Face está creando herramientas increíbles para NLP.


In [12]:
# 2) Generación de texto
from transformers import AutoTokenizer, AutoModelForCausalLM

# Cargamos el modelo y el tokenizer
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Texto de entrada en inglés y en español
prompt1 = "Once upon a time"
prompt2 = 'Erasé una vez'

# Tokenización
inputs_en = tokenizer.encode(prompt1, return_tensors='pt', max_length=512, truncation=True)
inputs_es = tokenizer.encode(prompt2, return_tensors='pt', max_length=512, truncation=True)

# Generación de texto
outputs_en = model.generate(inputs_en, max_length=512, num_return_sequences=1, do_sample=True)
outputs_es = model.generate(inputs_es, max_length=512, num_return_sequences=1, do_sample=True)

generated_text_en = tokenizer.decode(outputs_en[0], skip_special_tokens=True)
generated_text_es = tokenizer.decode(outputs_es[0], skip_special_tokens=True)

print(f"Texto generado en inglés: {generated_text_en}")
print(f"Texto generado en español: {generated_text_es}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Texto generado en inglés: Once upon a time, there was only time to rise above the noise."

A "perfect storm" would be at hand when his world came crashing down, he realized. The world he had created should be the result of his training, and that this world was still not the result of his training. A dream, or a nightmare, he knew. The world would be lost, torn apart, and broken without an idea of where it truly was going. An eternity passed before nothing ever arrived.

He was left alone, his mind and body in a clearing filled with people or magic waiting to take over. A few people were able to hear his commands. They spoke their words, their names, and their messages. He could speak with no fear and no hesitation whatsoever.

It was just one word on a blank tablet where those of his clan might see through the fog of war as he spoke. The name the tablet had been given when he arrived, the place he had left it after his departure.

The words he spoke were a message. The last word he had

El resultado en español ha sido... inesperado.